# Kotlin 함수형 프로그래밍
## Java 8+, Python, JavaScript와 비교하며 배우는 Kotlin 함수형 프로그래밍

이 노트북에서는 Kotlin의 함수형 프로그래밍 기능을 학습합니다.

### 학습 내용
1. 람다 표현식
2. 고차 함수
3. 컬렉션 함수형 연산
4. 시퀀스와 지연 평가
5. 함수 합성과 파이프라인

## 1. 람다 표현식

람다는 이름 없는 함수로, 값처럼 다룰 수 있습니다. Python의 lambda, Java 8의 람다, JavaScript의 화살표 함수와 유사합니다.

In [2]:
// 기본 람다 표현식
val sum = { a: Int, b: Int -> a + b }
println("람다로 더하기: 5 + 3 = ${sum(5, 3)}")

// 타입 명시가 있는 람다
val greet: (String) -> String = { name -> "안녕하세요, ${name}님!" }
println(greet("Kotlin"))

// 단일 매개변수는 it으로 참조 가능
val isEven: (Int) -> Boolean = { it % 2 == 0 }
println("4는 짝수인가? ${isEven(4)}")
println("7은 짝수인가? ${isEven(7)}")

람다로 더하기: 5 + 3 = 8
안녕하세요, Kotlin님!
4는 짝수인가? true
7은 짝수인가? false


In [4]:
// 람다와 컬렉션
val numbers = listOf(1, 2, 3, 4, 5)

// map: 각 요소를 변환 (Python의 map, Java의 stream().map()과 유사)
val doubled = numbers.map { it * 2 }
println("원본: $numbers")
println("2배: $doubled")

// 명시적 매개변수명 사용
val tripled = numbers.map { number -> number * 3 }
println("3배: $tripled")

원본: [1, 2, 3, 4, 5]
2배: [2, 4, 6, 8, 10]
3배: [3, 6, 9, 12, 15]


In [5]:
// 여러 줄 람다
val longGreeting = { name: String, time: String ->
    when (time) {
        "morning" -> "좋은 아침입니다, ${name}님!"
        "evening" -> "좋은 저녁입니다, ${name}님!"
        else -> "안녕하세요, ${name}님!"
    }
}

println(longGreeting("개발자", "morning"))
println(longGreeting("학생", "evening"))
println(longGreeting("방문자", "afternoon"))

좋은 아침입니다, 개발자님!
좋은 저녁입니다, 학생님!
안녕하세요, 방문자님!


In [7]:
// 클로저: 외부 변수 캡처
var counter = 0

val increment = {
    counter++  // 외부 변수를 캡처하여 수정
    println("카운터: $counter")
}

increment()
increment()
increment()

카운터: 1
카운터: 2
카운터: 3


### 💡 핵심 포인트
- **Python의 lambda보다** 더 강력하고 다양한 문법 지원
- **Java 8 람다와 유사**하지만 더 간결한 문법
- **JavaScript의 화살표 함수처럼** 클로저 지원
- **it 키워드**로 단일 매개변수 간단히 참조

## 2. 고차 함수

함수를 매개변수로 받거나 함수를 반환하는 함수입니다.

In [ ]:
// 함수를 매개변수로 받는 고차 함수
fun calculate(a: Int, b: Int, operation: (Int, Int) -> Int): Int {
    return operation(a, b)
}

// 다양한 연산 정의
val add = { x: Int, y: Int -> x + y }
val multiply = { x: Int, y: Int -> x * y }
val subtract = { x: Int, y: Int -> x - y }

println("10 + 5 = ${calculate(10, 5, add)}")
println("10 * 5 = ${calculate(10, 5, multiply)}")
println("10 - 5 = ${calculate(10, 5, subtract)}")

// 인라인 람다 사용
println("10 / 5 = ${calculate(10, 5) { x, y -> x / y }}")

In [8]:
// 함수를 반환하는 고차 함수 (커링과 유사)
fun createMultiplier(factor: Int): (Int) -> Int {
    return { number -> number * factor }
}

val double = createMultiplier(2)
val triple = createMultiplier(3)
val quadruple = createMultiplier(4)

println("7의 2배: ${double(7)}")
println("7의 3배: ${triple(7)}")
println("7의 4배: ${quadruple(7)}")

// 리스트에 적용
val numbers = listOf(1, 2, 3, 4, 5)
println("모든 수를 3배로: ${numbers.map(triple)}")

7의 2배: 14
7의 3배: 21
7의 4배: 28
모든 수를 3배로: [3, 6, 9, 12, 15]


In [10]:
// 확장 함수와 고차 함수의 조합
fun String.processText(transform: (String) -> String): String {
    return transform(this)
}

val text = "hello kotlin"

// 다양한 변환 적용
println("대문자: ${text.processText ({ it.uppercase() }) }")
println("역순: ${text.processText { it.reversed() }}")
println("첫 글자 대문자: ${text.processText { it.capitalize() }}")
println("단어별 대문자: ${text.processText { 
    it.split(" ").joinToString(" ") { word -> word.capitalize() }
}}")

대문자: HELLO KOTLIN
역순: niltok olleh
첫 글자 대문자: Hello kotlin
단어별 대문자: Hello Kotlin


In [15]:
// 실용적인 고차 함수 예제: 재시도 로직
fun <T> retry(times: Int, block: () -> T?): T? {
    repeat(times) { attempt ->
        val result = block()
        if (result != null) {
            println("성공! (시도 ${attempt + 1}회)")
            return result
        }
        println("실패... (시도 ${attempt + 1}회)")
    }
    return null
}

// 시뮬레이션: 50% 확률로 성공
val result = retry(5) {
    if (Math.random() > 0.5) "데이터 로드 성공" else null
}

println("최종 결과: $result")

실패... (시도 1회)
실패... (시도 2회)
성공! (시도 3회)
최종 결과: 데이터 로드 성공


### 💡 핵심 포인트
- **일급 함수**: 함수를 값처럼 다룰 수 있음
- **커링과 부분 적용** 패턴 구현 가능
- **전략 패턴**을 함수로 간단히 구현
- **코드 재사용성**과 모듈화 향상

## 3. 컬렉션 함수형 연산

Kotlin은 컬렉션을 다루는 풍부한 함수형 API를 제공합니다.

In [17]:
// 기본 컬렉션 연산
val numbers = listOf(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
println("원본: $numbers")

// filter: 조건에 맞는 요소만 선택
val evenNumbers = numbers.filter { it % 2 == 0 }
println("짝수: $evenNumbers")

// map: 각 요소를 변환
val squared = numbers.map { it * it }
println("제곱: $squared")

// filter + map 체이닝
val evenSquared = numbers
    .filter { it % 2 == 0 }
    .map { it * it }
println("짝수의 제곱: $evenSquared")

원본: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
짝수: [2, 4, 6, 8, 10]
제곱: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
짝수의 제곱: [4, 16, 36, 64, 100]


In [19]:
// reduce와 fold
val numbers = listOf(1, 2, 3, 4, 5)

// reduce: 첫 번째 요소부터 시작
val sum = numbers.reduce { acc, n -> acc + n }
println("합계 (reduce): $sum")

val product = numbers.reduce { acc, n -> acc * n }
println("곱 (reduce): $product")

// fold: 초기값 지정 가능
val sumFrom100 = numbers.fold(100) { acc, n -> acc + n }
println("100부터 시작한 합계 (fold): $sumFrom100")

// 문자열 연결
val words = listOf("Kotlin", "is", "awesome")
val sentence = words.fold("Hello") { acc, word -> "$acc $word" }.trim()
println("문장: $sentence")

합계 (reduce): 15
곱 (reduce): 120
100부터 시작한 합계 (fold): 115
문장: Hello Kotlin is awesome


In [20]:
// 실용적인 예제: 사람 데이터 처리
data class Person(val name: String, val age: Int, val city: String)

val people = listOf(
    Person("김철수", 25, "서울"),
    Person("이영희", 30, "부산"),
    Person("박민수", 22, "서울"),
    Person("정지은", 28, "대구"),
    Person("최동욱", 35, "서울"),
    Person("강미나", 26, "부산")
)

// 필터링
val seoulPeople = people.filter { it.city == "서울" }
println("서울 거주자:")
seoulPeople.forEach { println("  - ${it.name} (${it.age}세)") }

// 매핑
val names = people.map { it.name }
println("\n이름 목록: $names")

// 평균 계산
val averageAge = people.map { it.age }.average()
println("평균 나이: %.1f세".format(averageAge))

서울 거주자:
  - 김철수 (25세)
  - 박민수 (22세)
  - 최동욱 (35세)

이름 목록: [김철수, 이영희, 박민수, 정지은, 최동욱, 강미나]
평균 나이: 27.7세


In [28]:
// 그룹핑과 정렬
val people = listOf(
    Person("김철수", 25, "서울"),
    Person("이영희", 30, "부산"),
    Person("박민수", 22, "서울"),
    Person("정지은", 28, "대구"),
    Person("최동욱", 35, "서울"),
    Person("강미나", 26, "부산")
)

// groupBy: 특정 기준으로 그룹화
val peopleByCity = people.groupBy { it.city }
println("도시별 그룹:")
peopleByCity.forEach { (city, residents) ->
    println("  $city: ${residents.map { it.name }}")
}

// sortedBy: 정렬
val sortedByAge = people.sortedBy { it.age }
println("\n나이순 정렬:")
sortedByAge.forEach { println("  ${it.name} (${it.age}세)") }

// sortedByDescending: 역순 정렬
val sortedByAgeDesc = people.sortedByDescending { it.age }
println("\n나이 역순:")
sortedByAgeDesc.take(3).forEach { println("  ${it.name} (${it.age}세)") }

{서울=[Person(name=김철수, age=25, city=서울), Person(name=박민수, age=22, city=서울), Person(name=최동욱, age=35, city=서울)], 부산=[Person(name=이영희, age=30, city=부산), Person(name=강미나, age=26, city=부산)], 대구=[Person(name=정지은, age=28, city=대구)]}

나이순 정렬:
  박민수 (22세)
  김철수 (25세)
  강미나 (26세)
  정지은 (28세)
  이영희 (30세)
  최동욱 (35세)

나이 역순:
  최동욱 (35세)
  이영희 (30세)
  정지은 (28세)


In [30]:
// 고급 컬렉션 연산
val numbers = listOf(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

// partition: 조건에 따라 두 그룹으로 분할
val (evens, odds) = numbers.partition { it % 2 == 0 }
println("짝수: $evens")
println("홀수: $odds")

// any, all, none
println("\n조건 검사:")
println("10보다 큰 수가 있는가? ${numbers.any { it > 10 }}")
println("모두 양수인가? ${numbers.all { it > 0 }}")
println("음수가 없는가? ${numbers.none { it < 0 }}")

// find, firstOrNull
val firstEven = numbers.find { it % 2 == 0 }
println("\n첫 번째 짝수: $firstEven")

val firstOver10 = numbers.firstOrNull { it > 10 }
println("10보다 큰 첫 번째 수: $firstOver10")

// flatMap
val nestedList = listOf(
    listOf(1, 2, 3),
    listOf(4, 5),
    listOf(6, 7, 8, 9)
)
val flattened = nestedList.flatMap { it }
println("\n평탄화: $flattened")

짝수: [2, 4, 6, 8, 10]
홀수: [1, 3, 5, 7, 9]

조건 검사:
10보다 큰 수가 있는가? false
모두 양수인가? true
음수가 없는가? true

첫 번째 짝수: 2
10보다 큰 첫 번째 수: null

평탄화: [1, 2, 3, 4, 5, 6, 7, 8, 9]


### 💡 핵심 포인트
- **Java Stream API와 유사**하지만 더 간결
- **Python의 리스트 컴프리헨션**보다 명시적
- **체이닝**으로 복잡한 데이터 처리 파이프라인 구성
- **불변성**: 원본 컬렉션은 변경되지 않음

## 4. 시퀀스와 지연 평가

시퀀스는 지연 평가를 통해 대용량 데이터를 효율적으로 처리합니다.

In [31]:
// 즉시 평가 vs 지연 평가
val numbers = (1..1000000).toList()

// 즉시 평가: 각 단계마다 새로운 리스트 생성
val eagerResult = numbers
    .filter { it % 2 == 0 }    // 500,000개 리스트 생성
    .map { it * it }           // 500,000개 리스트 생성
    .take(5)                   // 5개 리스트 생성

println("즉시 평가 결과: $eagerResult")

// 지연 평가: 필요한 만큼만 계산
val lazyResult = numbers.asSequence()
    .filter { it % 2 == 0 }    // 연산만 기록
    .map { it * it }           // 연산만 기록
    .take(5)                   // 연산만 기록
    .toList()                  // 실제 계산 수행

println("지연 평가 결과: $lazyResult")

즉시 평가 결과: [4, 16, 36, 64, 100]
지연 평가 결과: [4, 16, 36, 64, 100]


In [34]:
// 무한 시퀀스
val infiniteSequence = generateSequence(1) { it + 1 }
println(infiniteSequence.take(5).toList())

// 3의 배수 중 처음 10개
val multiplesOf3 = infiniteSequence
    .filter { it % 3 == 0 }
    .take(10)
    .toList()

println("3의 배수 (처음 10개): $multiplesOf3")

// 제곱수 생성
val squares = generateSequence(1) { it + 1 }
    .map { it * it }
    .takeWhile { it < 100 }
    .toList()

println("100 미만의 제곱수: $squares")

[1, 2, 3, 4, 5]
3의 배수 (처음 10개): [3, 6, 9, 12, 15, 18, 21, 24, 27, 30]
100 미만의 제곱수: [1, 4, 9, 16, 25, 36, 49, 64, 81]


In [35]:
// 피보나치 수열 생성
fun fibonacci(): Sequence<Int> = sequence {
    var a = 0
    var b = 1
    
    yield(a)  // 첫 번째 값 반환
    yield(b)  // 두 번째 값 반환
    
    while (true) {
        val next = a + b
        yield(next)  // 다음 값 반환
        a = b
        b = next
    }
}

// 피보나치 수열의 처음 15개
val fib = fibonacci().take(15).toList()
println("피보나치 수열: $fib")

// 1000 미만의 피보나치 수
val fibUnder1000 = fibonacci()
    .takeWhile { it < 1000 }
    .toList()
println("1000 미만 피보나치: $fibUnder1000")

피보나치 수열: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]
1000 미만 피보나치: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]


In [36]:
// 시퀀스 연산의 실행 순서 확인
println("=== 즉시 평가 ===")
(1..5).toList()
    .filter { 
        println("filter: $it")
        it % 2 == 0 
    }
    .map { 
        println("map: $it")
        it * it 
    }

println("\n=== 지연 평가 ===")
(1..5).asSequence()
    .filter { 
        println("filter: $it")
        it % 2 == 0 
    }
    .map { 
        println("map: $it")
        it * it 
    }
    .toList()  // 터미널 연산

=== 즉시 평가 ===
filter: 1
filter: 2
filter: 3
filter: 4
filter: 5
map: 2
map: 4

=== 지연 평가 ===
filter: 1
filter: 2
map: 2
filter: 3
filter: 4
map: 4
filter: 5


[4, 16]

### 💡 핵심 포인트
- **Java Stream과 유사**한 지연 평가
- **Python의 generator**와 비슷한 개념
- **메모리 효율적**: 중간 컬렉션 생성 없음
- **무한 시퀀스** 처리 가능

## 5. 함수 합성과 파이프라인

함수들을 조합하여 더 복잡한 연산을 만들 수 있습니다.

In [38]:
// 기본 함수들
val trim = { s: String -> s.trim() }
val uppercase = { s: String -> s.uppercase() }
val addPrefix = { s: String -> "PREFIX: $s" }
val addSuffix = { s: String -> "$s :SUFFIX" }

// 함수 합성
fun <A, B, C> compose(f: (B) -> C, g: (A) -> B): (A) -> C = { x -> f(g(x)) }

val processText = compose(compose(addPrefix, uppercase), trim)
println(processText("  hello world  "))

// 여러 함수 합성
val fullProcess = compose(addSuffix, compose(addPrefix, compose(uppercase, trim)))
println(fullProcess("  kotlin  "))

PREFIX: HELLO WORLD
PREFIX: KOTLIN :SUFFIX


In [39]:
// 파이프라인 스타일 (더 읽기 쉬운 방식)
infix fun <A, B> ((A) -> B).then(next: (B) -> B): (A) -> B = { x -> next(this(x)) }

val pipeline = trim then uppercase then addPrefix then addSuffix
println(pipeline("  kotlin functional  "))

// 여러 변환 파이프라인
val removeSpaces = { s: String -> s.replace(" ", "") }
val reverse = { s: String -> s.reversed() }

val complexPipeline = trim then removeSpaces then uppercase then reverse
println(complexPipeline("  hello kotlin  "))

PREFIX: KOTLIN FUNCTIONAL :SUFFIX
NILTOKOLLEH


In [53]:
// 실용적인 예제: 주문 처리 파이프라인
data class Order(val id: Int, val amount: Double, val status: String)

val orders = listOf(
    Order(1, 100.0, "completed"),
    Order(2, 250.0, "pending"),
    Order(3, 150.0, "completed"),
    Order(4, 300.0, "cancelled"),
    Order(5, 200.0, "completed"),
    Order(6, 175.0, "pending")
)

// 완료된 주문의 총액
val totalCompleted = orders
    .filter { it.status == "completed" }
    .map { it.amount }
    .reduce { acc, amount -> acc + amount }

println("완료된 주문 총액: $$totalCompleted")

// 10% 할증 후 $150 초과 주문
val premiumOrders = orders
    .asSequence()
    .filter { it.status == "completed" }
    .map { it.copy(amount = it.amount * 1.1) }
    .map { it.copy(amount = it.amount.roundToInt().toDouble()) } // 두 번째 map: 소수점 이하 제거
    .filter { it.amount > 150 }
    .toList()

println("\n프리미엄 주문 (10% 할증 후 $150 초과):")
premiumOrders.forEach { println("  주문 #${it.id}: $${it.amount}") }

완료된 주문 총액: $450.0

프리미엄 주문 (10% 할증 후 $150 초과):
  주문 #3: $165.0
  주문 #5: $220.0


In [54]:
// 상태별 통계
val orderStats = orders
    .groupBy { it.status }
    .mapValues { (_, orders) -> 
        mapOf(
            "count" to orders.size,
            "total" to orders.sumOf { it.amount },
            "average" to orders.map { it.amount }.average()
        )
    }

println("\n주문 상태별 통계:")
orderStats.forEach { (status, stats) ->
    println("$status:")
    println("  개수: ${stats["count"]}")
    println("  총액: $${stats["total"]}")
    println("  평균: $%.2f".format(stats["average"]))
}

// 파티션을 사용한 분류
val (completed, notCompleted) = orders.partition { it.status == "completed" }
println("\n완료된 주문: ${completed.size}개")
println("미완료 주문: ${notCompleted.size}개")


주문 상태별 통계:
completed:
  개수: 3
  총액: $450.0
  평균: $150.00
pending:
  개수: 2
  총액: $425.0
  평균: $212.50
cancelled:
  개수: 1
  총액: $300.0
  평균: $300.00

완료된 주문: 3개
미완료 주문: 3개


### 💡 핵심 포인트
- **함수 합성**으로 복잡한 변환을 단순하게 표현
- **파이프라인**으로 데이터 흐름을 명확하게 표현
- **선언적 프로그래밍**: 무엇을 할지 기술
- **재사용 가능한** 작은 함수들의 조합

## 실습 과제

함수형 프로그래밍 개념을 활용하여 학생 성적 분석 시스템을 구현해보세요.

In [76]:
// 과제: 학생 성적 분석 시스템
data class Student(val name: String, val scores: Map<String, Int>)

val students = listOf(
    Student("김철수", mapOf("수학" to 85, "영어" to 92, "과학" to 78)),
    Student("이영희", mapOf("수학" to 95, "영어" to 88, "과학" to 91)),
    Student("박민수", mapOf("수학" to 73, "영어" to 79, "과학" to 82)),
    Student("정지은", mapOf("수학" to 88, "영어" to 95, "과학" to 87))
)

// TODO: 다음 기능들을 구현하세요
// 1. 각 학생의 평균 점수 계산
students.forEach { student ->
    val averageScore = student.scores.values.average()
    println("${student.name}의 평균 점수: %.2f".format(averageScore))
}
println("")

// 2. 과목별 최고 점수 학생 찾기
val subjects = listOf("수학", "영어", "과학")

subjects.forEach { subject ->
    val topStudent = students.maxByOrNull { it.scores[subject] ?: 0 }
    println("$subject 최고 점수 학생: ${topStudent?.name} (${topStudent?.scores?.get(subject)})")
}
println("")

// 3. 평균 80점 이상인 우수 학생 필터링
students.filter { it.scores.values.average() >= 80 }.forEach { student ->
    println("${student.name}의 평균 점수: %.2f".format(student.scores.values.average()))
}
println("")

// 4. 전체 학생의 과목별 평균 점수
subjects.forEach { subject ->
    val averageScore = students.map { it.scores[subject] ?: 0 }.average()
    println("$subject 평균 점수: %.2f".format(averageScore))
}
println("")


// 5. 성적순으로 학생 순위 매기기
students.sortedByDescending { it.scores.values.average() }.forEachIndexed { index, student ->
    println("${index + 1}위: ${student.name} (평균: %.2f)".format(student.scores.values.average()))
}


// 여기에 코드를 작성하세요
// 6. 특정 과목의 성적 분포 (예: 수학)
val mathScores = students.map { it.scores["수학"] ?: 0 }
println("\n수학 성적 분포:")
mathScores.groupingBy { it }.eachCount().forEach { (score, count) ->
    println("점수 $score: ${count}명")
}

// 7. 학생 이름을 대문자로 변환하여 출력
students.map { it.name.uppercase() }.forEach { name ->
    println("대문자 이름: $name")
}

// 8. 학생 이름과 평균 점수를 매핑하여 출력
val studentAverages = students.map { it.name to it.scores.values.average() }

studentAverages.forEach { (name, average) ->
    println("학생: $name, 평균 점수: %.2f".format(average))
}

// 9. 학생 이름과 최고 과목 점수를 매핑하여 출력
val studentTopScores = students.map { student ->
    val topSubject = student.scores.maxByOrNull { it.value }
    "${student.name}: ${topSubject?.key} (${topSubject?.value})"
}.forEach { println(it) }









김철수의 평균 점수: 85.00
이영희의 평균 점수: 91.33
박민수의 평균 점수: 78.00
정지은의 평균 점수: 90.00

수학 최고 점수 학생: 이영희 (95)
영어 최고 점수 학생: 정지은 (95)
과학 최고 점수 학생: 이영희 (91)

김철수의 평균 점수: 85.00
이영희의 평균 점수: 91.33
정지은의 평균 점수: 90.00

수학 평균 점수: 85.25
영어 평균 점수: 88.50
과학 평균 점수: 84.50

1위: 이영희 (평균: 91.33)
2위: 정지은 (평균: 90.00)
3위: 김철수 (평균: 85.00)
4위: 박민수 (평균: 78.00)

수학 성적 분포:
점수 85: 1명
점수 95: 1명
점수 73: 1명
점수 88: 1명
대문자 이름: 김철수
대문자 이름: 이영희
대문자 이름: 박민수
대문자 이름: 정지은
학생: 김철수, 평균 점수: 85.00
학생: 이영희, 평균 점수: 91.33
학생: 박민수, 평균 점수: 78.00
학생: 정지은, 평균 점수: 90.00
김철수: 영어 (92)
이영희: 수학 (95)
박민수: 과학 (82)
정지은: 영어 (95)
